<a href="https://colab.research.google.com/github/efearase/RL_with_sentiment/blob/main/features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from moexalgo import Ticker
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calculate_RSI(data, period=14):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))

    return RSI

def calculate_MACD(data, short_period=12, long_period=26, signal_period=9):
    short_EMA = data.ewm(span=short_period, adjust=False).mean()
    long_EMA = data.ewm(span=long_period, adjust=False).mean()

    MACD = short_EMA - long_EMA
    signal_line = MACD.ewm(span=signal_period, adjust=False).mean()
    histogram = MACD - signal_line

    return MACD

In [3]:
names = pd.read_csv('Export_ru_securities-list_20240129.csv', encoding='windows-1251', delimiter=';', quotechar='"')
codes = names[((names['LIST_SECTION']=='Первый уровень')) & (names['SUPERTYPE']=='Акции')]['TRADE_CODE'].dropna()

df_list = []

for code in codes:
    data = Ticker(code).candles(date='2022-03-20', till_date='today', period='1D')

    data['RSI'] = calculate_RSI(data['close'])
    data['MACD']= calculate_MACD(data['close'])

    data['Symbols'] = code

    data.rename(columns={'close': 'Adj Close', 'high': 'High', 'low': 'Low', 'open': 'Open', 'volume': 'Volume', 'begin':'Date'}
                , inplace=True)

    df_list.append(data)

panel_data = pd.concat(df_list, ignore_index=True)

panel_data.sort_values(by='Date', inplace=True)
panel_data.drop(columns = ['value', 'end'], inplace=True)
panel_data.to_csv('stocks_data/stocks_panel_data.csv', index=False)

In [4]:
panel_data

,Open,Adj Close,High,Low,Volume,Date,RSI,MACD,Symbols
0,5.1000,5.5340,5.66300,5.100000,3.372190e+07,2022-03-24,NaN,0.000000,CBOM
1987,0.0181,0.0190,0.02177,0.015995,1.303864e+11,2022-03-24,NaN,0.000000,VTBR
7005,251.4100,258.5100,278.90000,232.000000,8.587012e+07,2022-03-24,NaN,0.000000,GAZP
17236,130.0000,135.4000,157.95000,130.000000,1.806938e+07,2022-03-24,NaN,0.000000,SBERP
20343,0.6600,0.6815,0.69800,0.660000,6.837960e+08,2022-03-24,NaN,0.000000,HYDR
...,...,...,...,...,...,...,...,...,...
18826,18.9100,19.0200,19.02500,18.880000,4.663500e+06,2024-03-26,44.820717,0.427940,SVCB
8020,15156.0000,15130.0000,15314.00000,15106.000000,3.110100e+04,2024-03-26,57.348243,-76.325582,GMKN
16727,79.2000,79.4000,79.55000,78.550000,2.497600e+05,2024-03-26,47.315436,1.596217,RTKMP
5482,308.2000,307.1100,308.74000,304.800000,1.992500e+05,2024-03-26,62.002540,5.522067,BSPB


In [5]:
data

,Open,Adj Close,High,Low,value,Volume,Date,end,RSI,MACD,Symbols
0,313.0,329.0,369.9,300.4,6.045241e+08,1767605.0,2022-03-24,2022-03-24 23:59:59,NaN,0.000000,TATNP
1,349.0,318.5,349.0,315.0,1.948517e+08,605574.0,2022-03-25,2022-03-25 23:59:59,NaN,-0.837607,TATNP
2,326.0,318.0,327.0,303.0,1.785893e+08,569398.0,2022-03-28,2022-03-28 23:59:59,NaN,-1.524192,TATNP
3,321.8,336.7,344.0,313.0,3.975852e+08,1188152.0,2022-03-29,2022-03-29 23:59:59,NaN,-0.553008,TATNP
4,336.9,351.8,356.4,330.1,3.140808e+08,894241.0,2022-03-30,2022-03-30 23:59:59,NaN,1.418753,TATNP
...,...,...,...,...,...,...,...,...,...,...,...
503,703.3,696.2,706.4,678.2,1.706275e+09,2470143.0,2024-03-20,2024-03-20 23:59:59,50.632911,3.711379,TATNP
504,697.5,701.7,703.9,691.1,4.039225e+08,579174.0,2024-03-21,2024-03-21 23:59:59,46.206897,2.106546,TATNP
505,704.0,690.0,708.9,687.2,5.296558e+08,759922.0,2024-03-22,2024-03-22 23:59:59,39.062500,-0.108141,TATNP
506,690.2,686.5,696.1,677.3,6.297554e+08,920811.0,2024-03-25,2024-03-25 23:59:59,31.501472,-2.121265,TATNP
